# Libraries

In [1]:
import numpy as np
import scipy.constants as sc # this gives all constants in SI units

#  Constants

In [2]:
iota = complex(0,1)
pi = sc.pi
hbar = sc.hbar
speed_of_light = sc.c
amu = sc.atomic_mass
micro_meters = sc.micro
nano_kelvin = sc.nano
kilohertz   = sc.kilo

# Main parameters

#### Number of atoms
$N = 2\times10^4$,

#### Angular momentum numbers
$L_p = 20$, $l = 130$

#### Decay rates
Damping rate of each condensate, $\gamma_m \sim 2\pi\times0.0167$ Hz, Cavity decay rate, $\gamma_0 = 2\pi\times 1$ KHz, 

#### Radius of toroidal trap
$R = 10$ $\mu$m,

#### Mass of each atom
$m = 23$ amu, $(1\text{ amu } = 1.66054\times10^{-27}$ Kg) 


#### Harmonic trapping frequencies
$\omega_\rho =\omega_z= 2\pi\times 840$ Hz

#### Detunings
Decay rate of the sodium D line, $\gamma_a/2\pi \approx 9.8$ MHz,  detuning of optical frequency from the atomic transition, $\Delta_a/2\pi = 400\times \gamma_a$

The drive laser is Blue detuned far from the ground-to-excited state atomic transition and therefore, interaction between drive laser and atoms is weak.

#### Other parameters
$g_a = 2\pi\times 0.36$ MHz


#### Field amplitude
$\varepsilon_{p(l)} = \sqrt{\frac{\mu\kappa\mathcal{P}}{\hbar \omega_{p(l)}}}$

#### Linearity condition
$$
\hbar U_0|\alpha_s|^2 <<\frac{\hbar^2 L_p^2}{2mR^2} + 2\hbar\tilde{g}N
$$

# All parameters (SI)

####  Toroidal trapping potential parameters

In [3]:
# Radius of Toroidal trap
Radius = 10*sc.micro          # micrometers in terms of meters

# Frequencies related to the trapping potential in Hz
omega_rho_0 = 2*pi*840
omega_z_0   = omega_rho_0


#### Ring BEC parameters

In [4]:
# Mass of atom
mass = 23*amu

#
a_Na = 0.1*sc.nano

# Moment of inertia of the rotating superfluid
moment_of_inertia = mass*Radius**2

# Number of atoms in the BEC
Na = 2e4

# atom-field coupling strength
g_a = 2*pi*0.36*sc.mega

# decay rate of the sodium D line
gamma_a = 2*pi*9.8*sc.mega

# saturation intensity 
I_sat = 10e-3/(1e-2)**2      # watts per meter square

# matter wave mode decay rate
gamma_m_0 = 2*pi*(1/60)     # life time of the BEC is taken to be 60 seconds

# Winding number of the BEC wavefunction
Lp = 20

# detuning of the drive field from the resonance of atomic transition
delta_a = 300*gamma_a


#### One dimentionality condition

$$
N<\frac{4\sqrt{\pi}R}{3a_{Na}}\left(\frac{\omega_\rho}{\omega_z}\right)^{\frac{1}{2}}
$$

In [5]:
RHS = (4*np.sqrt(pi)*Radius/(3*a_Na))*np.sqrt(omega_rho_0/omega_z_0)

if RHS/Na >1:
    print("One dimensionality satisfied")
else:
    print("One dimensionality not satisfied")

print("RHS is greater then LHS by,", np.round(RHS/Na, 3), "times")

One dimensionality satisfied
RHS is greater then LHS by, 11.816 times


#### Fabri Perot cavity parameters

In [6]:
# Cavity decay rate
kappa_0   = 2*pi*sc.kilo

# cavity field coupling rate
mu = 0.5

#### Coupling constants

$$
\tilde{\Delta} = \Delta_0 - \frac{U_0 N}{2\sqrt{2}}\quad (\Delta_0 = \omega_{drive} - \omega_{cav})
$$
$$
U_0 = \frac{g^2_a}{\Delta_a},\; \tilde{g} = \frac{g}{4\pi\hbar},\; G = \frac{U_0\sqrt{N}}{2\sqrt{2}}
$$

In [7]:
# U_0
U_0 = g_a**2/delta_a

# Effective optomechanical coupling constant
G_0 = U_0*np.sqrt(Na)/(2*np.sqrt(2))


#### Drive laser parameters

In [8]:
# Drive laser wavelength
lambda_l  = 589*sc.nano      

# Drive laser frequency
omega_l0 = 2*pi*speed_of_light/lambda_l + delta_a

# OAM index of the drive LG mode
OAM_index = 130

#### Atom-atom interaction strength

In [9]:
# g_tilde
g_tilde_0 = omega_rho_0*a_Na/(2*pi*Radius)


#### BEC matter wave frequencies

In [10]:
def BEC_frequencies(ell, Lp):
    global mass, Radius
    from scipy import constants as sc
    
    # Given parameters
    Temp = 20*sc.nano           # Temperature in Kelvin (10 nano Kelvin)

    # Moment of inertia
    I = mass * Radius**2

    # Frequency calculation
    omega_Lp = (sc.hbar * Lp**2) / (2 * I)
    omega_c  = (sc.hbar * (Lp + 2 * ell)**2) / (2 * I)
    omega_d  = (sc.hbar * (Lp - 2 * ell)**2) / (2 * I)

    # Number of phonons using Bose-Einstein distribution
    n_phonons_c = 1 / (np.expm1(sc.hbar * omega_c / (sc.Boltzmann * Temp)))   # expm1 means exp(something) - 1
    n_phonons_d = 1 / (np.expm1(sc.hbar * omega_d / (sc.Boltzmann * Temp)))

    return omega_Lp, omega_c, omega_d, n_phonons_c,  n_phonons_d

omega_Lp0, omega_c0, omega_d0, nth_c, nth_d = BEC_frequencies(OAM_index, Lp)

# Other frequencies
capital_omega_0 = (omega_c0 + omega_d0 + 4*g_tilde_0*Na)/((omega_c0 + 4*g_tilde_0*Na)*(omega_d0 + 4*g_tilde_0*Na) - 4*g_tilde_0**2*Na**2)


#### Critical detuning
$$
\tilde{\Delta} = -\frac{\sqrt{3}\kappa}{2}
$$

In [11]:
critical_detuning_0 = -np.sqrt(3)*kappa_0/2

#### Critical Power
Critical power here refers to the power of the drive laser at the critical detuning, $\tilde{\Delta} = -\frac{\sqrt{3}\kappa}{2}$

$$
\mathcal{P}_{\text{cr}} = \frac{\hbar\omega_l\kappa^2}{3\sqrt{3}G^2\Omega\mu}
$$

In [12]:
critical_power = hbar*omega_l0*kappa_0**2/(3*np.sqrt(3)*G_0**2*capital_omega_0*mu)

# Parameters normalized w.r.t $\gamma_0$
In this code, $\gamma_0$ is denoted by $\kappa$

In [13]:
######################################
#         Normalized parameters
######################################

# Cavity decay rate
kappa = kappa_0/kappa_0

# Damping rate of each condensate
gamma_m = gamma_m_0/kappa_0

# Harmonic trapping frequencies
omega_rho = omega_rho_0/kappa_0
omega_z   = omega_z_0/kappa_0

# Other parameters
G = G_0/kappa_0

U = U_0/kappa_0

# Matter wave frequencies
omega_c = omega_c0/kappa_0
omega_d = omega_d0/kappa_0

# g_tilde
g_tilde = g_tilde_0/kappa_0

# critical detuning
critical_detuning = critical_detuning_0/kappa_0

# small delta
small_delta   = omega_c + 4*g_tilde*Na     # small delta = omega_p - omega_l

# drive frequencies
omega_l = omega_l0/kappa_0

# Cavity resonance frequency
omega_cav_eff = omega_l + small_delta


# Probe frequency
omega_p = omega_cav_eff

# effective detunings
delta_tilde   = -small_delta

# omega_bar
capital_omega_bar = (omega_c + omega_d + 4*g_tilde*Na)/((omega_c + 4*g_tilde*Na)*(omega_d + 4*g_tilde*Na) - 4*g_tilde**2*Na**2)

In [14]:
# Drive laser power
power_l  = 7e4*critical_power

# Power of signal
power_p = power_l*1e-2

# Field amplitudes
eta_l = np.sqrt(mu*kappa*power_l/(hbar*omega_l))/kappa_0
eta_p = eta_l*1e-2


In [15]:
print("drive power = ", power_l)
print("eta_l and eta_p : ", eta_l, eta_p)

drive power =  8.584293215184512e-10
eta_l and eta_p :  450.0547675537846 4.500547675537846


In [16]:
print("omega_c = ", np.round(omega_c, 3))
print("omega_d = ", np.round(omega_d, 3))
print("omega_c-omega_d = ", np.round(omega_c-omega_d,3))


omega_c =  172.268
omega_d =  126.564
omega_c-omega_d =  45.704


In [17]:
g_prime = 2*g_tilde*Na

print("g_prime/gamma_0 = ", np.round(g_prime,3))

g_prime/gamma_0 =  0.053


In [18]:
print("number of thermal photons in atomic modes c and d: ", nth_c, nth_d)

number of thermal photons in atomic modes c and d:  2.963996425317431e-180 1.264118188079506e-132
